In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [16]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(5, 110, 5)
non_lin = [2]
non_lin_dict = {1: "sigmoid", 2: "relu"}
alpha = 0

#empty dictionary
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c])) 
        
        #y_train[y_train < 0] = 0
        #y_test[y_test < 0 ] = 0
        
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, lr = 1e-2)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 100)

        w, o, s  = model.get_params()
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                 100%
[                     ]

testing non-linearity:  relu ... testing  5  pc's...
new best model for cell  517442753 . r2 test:  -0.0441007965301
new best model for cell  517444738 . r2 test:  -0.00582692522765
new best model for cell  517443331 . r2 test:  0.0462302901453
new best model for cell  517442436 . r2 test:  0.0135116897042
new best model for cell  517445575 . r2 test:  0.0198250235879
new best model for cell  517444426 . r2 test:  -0.0266025339989
new best model for cell  517442471 . r2 test:  -0.00215595571986
new best model for cell  517443181 . r2 test:  0.0488096711097
new best model for cell  517443087 . r2 test:  0.0271518942021
new best model for cell  517446833 . r2 test:  0.0165817674112
new best model for cell  517443827 . r2 test:  -0.00112441996116
new best model for cell  517443924 . r2 test:  -0.00977146658756
new best model for cell  517443957 . r2 test:  -0.0136677861697
new best model for cell  517442722 . r2 test:  -0.0133505466714
new best model for cell  517446488 . r2 test:  -0.023

[#                    ] | ETA: 03:20:18

new best model for cell  517442753 . r2 test:  -0.014687618978
new best model for cell  517444738 . r2 test:  0.00298468283524
new best model for cell  517444426 . r2 test:  -0.0101529963895
new best model for cell  517442471 . r2 test:  0.0224928116491
new best model for cell  517443827 . r2 test:  0.0172765943257
new best model for cell  517443924 . r2 test:  0.0307861865481
new best model for cell  517442722 . r2 test:  0.0240453494707
new best model for cell  517446488 . r2 test:  -0.0122962237381
new best model for cell  517442675 . r2 test:  0.0131410710983
new best model for cell  517446554 . r2 test:  -0.0148506668881
new best model for cell  517443571 . r2 test:  -0.00805290122262
testing non-linearity:  relu ... testing  15  pc's...


[###                  ] | ETA: 04:25:05

...

[###################  ] | ETA: 01:19:58

testing non-linearity:  relu ... testing  100  pc's...


[#################### ] | ETA: 00:40:23

new best model for cell  517443827 . r2 test:  0.06210419927
testing non-linearity:  relu ... testing  105  pc's...


[#####################] | ETA: 00:00:00
Total time elapsed: 14:18:17


In [8]:
pkl_file = open('./boc/formatted/LNE_relu', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()